In [1]:
import torch

In [2]:
torch.__version__

'2.6.0+cu124'

In [5]:
import transformers
from transformers import AutoTokenizer

In [6]:
 tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [36]:
sentence = "I skip across the"

In [37]:
input_ids = tokenizer(sentence,return_tensors='pt').input_ids

In [38]:
input_ids

tensor([[   40, 14267,  1973,   262]])

In [39]:
tokenizer.decode(37534)

'Prep'

In [40]:
for token_id in input_ids[0]:
    print(tokenizer.decode(token_id))

I
 skip
 across
 the


In [41]:
from transformers import AutoModelForCausalLM

In [42]:
gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")

In [43]:
outputs = gpt2(input_ids)

In [44]:
outputs.logits

tensor([[[-39.3084, -39.0100, -41.8374,  ..., -46.9337, -44.9074, -39.5149],
         [-78.0579, -78.4766, -82.9907,  ..., -87.3913, -86.9834, -79.6007],
         [-73.0664, -74.0119, -78.5556,  ..., -81.0934, -82.5249, -75.8570],
         [-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209]]],
       grad_fn=<UnsafeViewBackward0>)

In [45]:
final_logits = gpt2(input_ids).logits[0,-1]

In [46]:
final_logits

tensor([-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209],
       grad_fn=<SelectBackward0>)

In [47]:
final_logits.argmax()

tensor(1627)

In [48]:
tokenizer.decode(final_logits.argmax())

' line'

In [49]:
top10_logits = torch.topk(final_logits,10)

In [50]:
for index in top10_logits.indices:
    print(tokenizer.decode(index))

 line
 street
 river
 room
 pond
 bridge
 border
 country
 road
 board


In [56]:
final_logits.softmax(dim=0)

tensor([5.5733e-07, 2.4777e-06, 2.5463e-07,  ..., 2.7615e-07, 2.7765e-08,
        2.3236e-06], grad_fn=<SoftmaxBackward0>)

In [57]:
top10 = torch.topk(final_logits.softmax(dim=0),10)

In [58]:
top10

torch.return_types.topk(
values=tensor([0.0361, 0.0269, 0.0222, 0.0188, 0.0183, 0.0173, 0.0168, 0.0166, 0.0160,
        0.0094], grad_fn=<TopkBackward0>),
indices=tensor([ 1627,  4675,  7850,  2119, 16723,  7696,  4865,  1499,  2975,  3096]))

In [60]:
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index)} -- {value.item():.1%}")

 line -- 3.6%
 street -- 2.7%
 river -- 2.2%
 room -- 1.9%
 pond -- 1.8%
 bridge -- 1.7%
 border -- 1.7%
 country -- 1.7%
 road -- 1.6%
 board -- 0.9%


In [64]:
output_ids = gpt2.generate(input_ids,max_new_tokens=20, repetition_penalty=1.5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [65]:
decoded_text = tokenizer.decode(output_ids[0])

In [67]:
print(decoded_text)

I skip across the line to a place where I can see it.

The next time you're in town,


In [69]:
output_ids = gpt2.generate(input_ids,max_new_tokens=20, do_sample=True, top_k=5)
decoded_text = tokenizer.decode(output_ids[0])
print(decoded_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I skip across the line.

I don't know why.

I think I'm not the first person


In [74]:
output_ids = gpt2.generate(input_ids,max_new_tokens=20, do_sample=True, temperature=1.0)
decoded_text = tokenizer.decode(output_ids[0])
print(decoded_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I skip across the map when you make your way through the forest in the way that I can, and if you just


In [76]:
from transformers import pipeline

In [77]:
pipe = pipeline("text-generation", model="openai-community/gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [78]:
pipe("I went to the store and bought a")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I went to the store and bought a pack and my husband was happy to walk me out. As for the customer service, I am very impressed to understand why so many people would prefer not to take their cards. It is the simplest way to get'}]